# extract articles and comments from .json

- output .pkl: from `article_text` and `processed_comment_text` to `articles.pkl` and `processd_comments.pkl` 
- from `articles.pkl` to `processed_articles.pkl`
- update `processed_comments.pkl` because the official `Data_curation_language.ipynb` has some problem

In [20]:
import os
import pandas as pd
import pickle

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Extract articles and process_comments

extract `article_text` and `process_comments_text` from `.json` files

In [2]:
# data_base_path = "../curated-source-dataset/english/unbiased"
data_base_path = "../curated-source-dataset/english"
json_list = []
for root, dirs, files in os.walk(data_base_path):
    for file in files:
        if file.endswith('.json'):
            json_list.append(os.path.join(root, file))
print("total number of json files:", len(json_list), '\nlist of json files:', json_list)

total number of json files: 17 
list of json files: ['../curated-source-dataset/english\\biased\\fb.json', '../curated-source-dataset/english\\biased\\foodbabe.json', '../curated-source-dataset/english\\biased\\foodrevolution.json', '../curated-source-dataset/english\\biased\\organicauthority.json', '../curated-source-dataset/english\\biased\\organicconsumers.json', '../curated-source-dataset/english\\unbiased\\Forum\\cafemom.json', '../curated-source-dataset/english\\unbiased\\Forum\\disqus.json', '../curated-source-dataset/english\\unbiased\\Forum\\quora.json', '../curated-source-dataset/english\\unbiased\\Forum\\reddit.json', '../curated-source-dataset/english\\unbiased\\Forum\\usmessageboard.json', '../curated-source-dataset/english\\unbiased\\newssites\\chicagotribune.json', '../curated-source-dataset/english\\unbiased\\newssites\\huffingtonpost.json', '../curated-source-dataset/english\\unbiased\\newssites\\latimes.json', '../curated-source-dataset/english\\unbiased\\newssites\\n

an example json file

In [5]:
df_eg = pd.read_json(json_list[1])
print(df_eg)
print(df_eg.dtypes)

                                        article_title  \
0   My Favorite Organic Non-GMO Snacks and Party F...   
1                             Is Organic Food A Scam?   
2                                 Organic Trip Tricks   
3   Full List of Non Organic Ingredients Allowed i...   
4   This New Label On Food Will Keep You From Gett...   
5   The Shocking Difference Between Organic & Non-...   
6   This Information Could Change Your Grocery Sho...   
7   Fast Food - The Quickest Organic Burrito You'l...   
8   After You Read This You'll Never Look At Milk ...   
9                                Is Subway Real Food?   
10             Chemical Warfare with "Natural Flavor"   
11                     How To Eat Organic On A Budget   
12  Monsanto Is Scrambling To Bury This Breaking S...   
13      How to Find the Safest Organic Infant Formula   
14                       What Is Trader Joe's Hiding?   

                            article_author        article_time  \
0   [{'article_author

In [3]:
# def extract_comment(df):
#     df_new = pd.DataFrame()
#     df_new['comment'] = ''
#     # print(df.shape)
#     df = df[(df.relevant == 1)]     # filter out not relevant articles
#     # print(df.shape)
#     for comments in df['comments']:
#         for comment in comments:
#             if comment['processed_comment_text'] != '':     # filter out blank comment
#                 df_new = df_new.append({'comment': comment['processed_comment_text'],}, ignore_index=True)
#     return df_new

def extract_comment(df):
    n_article = len(df)
    df = df[df['relevant'] == 1]
    n_article_rel = len(df)
    print("{:.1f}% of articles are relevant".format(n_article_rel / n_article * 100))
    comment_list = []
    for comments in df['comments']:
        for comment in comments:
            if comment["processed_comment_text"] != '':
                comment_list.append(comment["processed_comment_text"])
    df_comment = pd.DataFrame(comment_list, columns=['comment'])
    return df_comment

def extract_article_comment(df):
    n_article = len(df)
    df = df[df['relevant'] == 1]
    n_article_rel = len(df)
    rel_ratio = n_article_rel / n_article
    # print("relevant ratio: {:.1f}%".format(rel_ratio * 100))
    comment_list = []
    article_list = []
    for index, row in df.iterrows():
        # processed_article_text -> in separate step
        if row['article_text'] != '':
            article_list.append(row['article_text'].replace('\n', ' '))
        for comment in row['comments']:
            if comment["processed_comment_text"] != '':
                comment_list.append(comment["processed_comment_text"])
    df_article = pd.DataFrame(article_list, columns=['article'])
    df_comment = pd.DataFrame(comment_list, columns=['comment'])
    return df_article, df_comment, rel_ratio


In [9]:
df_eg_art, df_eg_com, _ = extract_article_comment(df_eg)
print(df_eg_art)
print(df_eg_com)

                                              article
0   I choose to eat organic food whenever possible...
1   Today I posted an article my cousin sent me on...
2   A large body of peer reviewed research links g...
3   One of the things I love to do every year is v...
4   My decisions are simple when I go grocery shop...
5   I find it disturbing that there are people who...
6   Subway is the single largest chain restaurant ...
7   I am so happy I don’t have to watch the news o...
8   The one tried and true worry I get about livin...
9   I am frankly shocked this information is not m...
10  I am honored to share this guest post by Charl...
11  The answer to that question is easy – A LOT. A...
                                                comment
0     I would NEVER take food/nutrition advise from ...
1     If you think FoodBabe is not a business orient...
2     You are funny Emma, if you know anything about...
3     I love how people knock down those that get pa...
4     Love you, Fo

In [10]:
print(df_eg_art.iloc[0].article)

I choose to eat organic food whenever possible because I believe it’s better for my health, the environment, and the farmers who grow our food. But not everyone agrees with me. There are a lot of people who think organic food is too expensive, a waste of money, and a scam. Do you feel that way? It’s no wonder people feel this way, with headlines like these… “Buying organic veggies at the supermarket is a waste of money” – Quartz “The USDA ‘Organic’ Label Misleads And Rips Off Consumers” – Forbes “Organic Foods Are Just A ‘Marketing Label” – Business Insider “Don’t Believe the (Organic) Hype” – NPR “Is organic food worth the higher price? Experts say no” – Portland Tribune One of the biggest perpetrators of these beliefs is Monsanto (and other big biotech companies like Syngenta and Bayer). Think about it: their best-selling products like Roundup and GMO seeds are banned on organic farms. If all farms were organic their biggest products would bite the dust! Any messaging that organic fo

In [4]:
%%time
df_article_list = []
df_comment_list = []
for file in json_list:
    df = pd.read_json(file)
    # df_new = extract_comment(df)
    df_arti, df_comm, rel_ratio = extract_article_comment(df)
    print(os.path.basename(file), df.shape, df_arti.shape, df_comm.shape)
    df_article_list.append(df_arti)
    df_comment_list.append(df_comm)
df_article = pd.concat(df_article_list, axis=0, ignore_index=True)
df_comment = pd.concat(df_comment_list, axis=0, ignore_index=True)

fb.json (5035, 13) (1464, 1) (77145, 1)
foodbabe.json (15, 10) (12, 1) (3693, 1)
foodrevolution.json (78, 10) (47, 1) (1323, 1)
organicauthority.json (66, 10) (43, 1) (0, 1)
organicconsumers.json (64, 10) (43, 1) (0, 1)
cafemom.json (88, 10) (24, 1) (329, 1)
disqus.json (42, 12) (33, 1) (7573, 1)
quora.json (1703, 10) (484, 1) (8634, 1)
reddit.json (256, 13) (63, 1) (8134, 1)
usmessageboard.json (382, 10) (0, 1) (1954, 1)
chicagotribune.json (2328, 10) (572, 1) (151, 1)
huffingtonpost.json (880, 10) (668, 1) (0, 1)
latimes.json (1522, 10) (270, 1) (14, 1)
nypost.json (106, 10) (33, 1) (0, 1)
nytimes.json (438, 10) (327, 1) (11567, 1)
usatoday.json (95, 10) (61, 1) (191, 1)
washingtonpost.json (1563, 12) (489, 1) (14771, 1)
Wall time: 11 s


In [5]:
print(df_article.shape)
print(df_comment.shape)
print(df_article.head())
print(df_comment.head())
# (4633, 1)
# (135479, 1)

(4633, 1)
(135479, 1)
                                             article
0  Supposed climate independent food research gro...
1  A form of what is now called regenerative agri...
2  Did you know the majority of olive oil sold in...
3  Even light doses of glyphosate can cause disea...
4  NEW REPORT: Natural climate solutions—includin...
                                             comment
0                   That was another administration.
1  Why is the ex CEO of Monsanto appointed head o...
2  because he greased the palms of our "elected" ...
3  the photo shown above is from a feed lot in Ca...
4  THAT IS NOT GRASS FED MEET! NO GRASS THERE! YO...


In [6]:
processed_path = "../processed"
os.makedirs(processed_path, exist_ok=True)
with open(os.path.join(processed_path, 'processed_comments.pkl'), 'wb') as f:
    pickle.dump(df_comment, f)
with open(os.path.join(processed_path, 'articles.pkl'), 'wb') as f:
    pickle.dump(df_article, f)

In [7]:
with open(os.path.join(processed_path, 'processed_comments.pkl'), 'rb') as f:
    df_comment = pickle.load(f)
df_comment
# 135479 rows × 1 columns

,comment
0,That was another administration.
1,Why is the ex CEO of Monsanto appointed head o...
2,"because he greased the palms of our ""elected"" ..."
3,the photo shown above is from a feed lot in Ca...
4,THAT IS NOT GRASS FED MEET! NO GRASS THERE! YO...
...,...
135474,"Recognizing the power of Branding, the Amish s..."
135475,Amish tend to avoid 'branding' anything they m...
135476,"The Amish community around Lancaster, PA love ..."
135477,Look for Organic Valley dairy products. You ca...


In [9]:
df_comment.comment.iloc[-21]

'Coincidentally, yesterday I just watched, "What the Dairy Industry does not Want You to Know" by Neal Barnard MD (https://www.youtube.com/watch?v=h3c_D0s391Q&amp;t=150s) and that sure makes a strong case against dairy of any kind, organic or not. And I grew up on a farm too! I will stick to buying Amish furniture I guess. there is a place in my town that is called, "Amish Woodcrafts".'

In [12]:
with open(os.path.join(processed_path, 'articles.pkl'), 'rb') as f:
    df_article = pickle.load(f)
df_article
# 4633 rows instead of 5887 rows when filter out article with ''

,article
0,Supposed climate independent food research gro...
1,A form of what is now called regenerative agri...
2,Did you know the majority of olive oil sold in...
3,Even light doses of glyphosate can cause disea...
4,NEW REPORT: Natural climate solutions—includin...
...,...
4628,"Attention, shoppers: Whole Foods Market is now..."
4629,Most of us don’t spend our days plowing fields...
4630,Bogus “organic” products may be reaching the U...
4631,"BLYTHEVILLE, ARK. — Clay Mayes slams on the br..."


In [14]:
print(df_article.iloc[4324].article)

With all the recent interest in locally grown, farm-fresh and organic foods, it’s no wonder the number of U.S. farmers markets has more than tripled in the past 15 years. There are now more than 7,175 across the country, according to the Agriculture Department . Buying local means products are picked at peak ripeness, providing the top freshness, flavor and nutrients. The foods are transported fewer miles and are coming directly from the people who grow them. And you can feel good about supporting your local economy and strengthening the local food system. Most farmers markets in our area open in April or May. So now is the time to get exploring, to discover new produce and fresh or potted herbs that your supermarket might not feature. For an interactive map of markets in the Washington area, go to washingtonpost.com/food . Feeling inspired? This salad recipe makes use of seasonal greens including peppery arugula and watercress, buttery bib lettuce and velvety mache (also known as lamb

# process article

https://gitlab.lrz.de/social-rom/datasets/organic-dataset/curated-source-dataset/-/blob/master/english/Data_curation_language.ipynb


In [42]:
%cd ../scripts/

C:\Project\group-1.3\scripts


In [11]:
from utils import filter_articles_pandas
import pickle
import regex as re

In [12]:
processed_path = "../processed"
with open(os.path.join(processed_path, 'articles.pkl'), 'rb') as f:
    df_article = pickle.load(f)

In [43]:
df_article.to_csv('../processed/articles.csv', sep='\t')

In [17]:
len(df_article) # 4633

4633

clean articles a little bit

In [21]:
%%time
# Loading dataset as pandas df

# Load your comments file as pandas dataframe 'df_article'. ENSURE it has a COLUMN named 'article'

print('Inital count: ', df_article.shape)

#Filtering relevant comments
article_filtered = filter_articles_pandas(df_article)
article_filtered.reset_index(drop=True, inplace=True)

#Mention the directory to save the output

#Exporting the filtered comment to text file
# file_output_txt_filename = directory + "/output/filtered_comments.txt" 
# json_filtered.to_csv(file_output_txt_filename, header=False, index=False, sep='\t', mode='w+')

print('Final count',article_filtered.shape)
print('completed')
# 4451 rows × 1 columns
# (4479, 1) after fixing no punctuation mark issue 

Inital count:  (4633, 1)
removing Devanagari...
do some hard-coded replacement...
removing url...
calculating filtering criteria...
Final count (4479, 1)
completed
Wall time: 1min 31s


In [25]:
#Testing the final filterd dataframe
# article_filtered.rename(columns={'tidy_article': 'article'}, inplace=True)
article_filtered.iloc[38].article

'Who loses when farmers go organic? Well, agrochemical companies that supply pesticides, herbicides and fungicides. '

In [26]:
df_article['article'][39]

'Who loses when farmers go organic? Well, agrochemical companies that supply pesticides, herbicides and fungicides. http://orgcns.org/2rVgITa'

In [27]:
with open(os.path.join(processed_path, 'processed_articles.pkl'), 'wb') as f:
    pickle.dump(article_filtered, f)

In [28]:
article_filtered['article'][4]

'NEW REPORT: Natural climate solutions—including regenerative farming and land management—could reduce emissions by 11.3 billion tons per year by 2030. '

In [44]:
article_filtered.to_csv("../processed/processed_articles.csv", sep='\t')

In [ ]:
# # Loading dataset as json files

# import os, json
# import pandas as pd
# import regex as re


# # provide folder path of json files
# path_to_json = #####
# json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# print('No. of files read: ', len(json_files))
# print('Names of files read: ', json_files)


# # we need both the json and an index number so use enumerate()
# for index, js in enumerate(json_files):
#     output_file = path_to_json + '/processed_' + js
#     with open(os.path.join(path_to_json, js), encoding='utf-8') as json_file:
#         json_text = json.load(json_file)
#         print('filename: ', js)
#         print('no. of rows in file: ', len(json_text))
        
#         i = 1
#         for data in json_text:           
#             for commentdata in data['comments']:
       
#                 commentdata['processed_comment_text'] = filter_comments_json(commentdata['comment_text']) if commentdata['comment_text'] != '' else commentdata['comment_text']

# #             print(i)                
#             i = i + 1
    
#         #Output will be saved as processed_filename.json in same folder
#         with open(output_file, 'w') as json_out_file:
#                 json.dump(json_text, json_out_file)
                
# print('completed')


# update `processed_comments`

In [29]:
# print(sys.path)
%cd ..

C:\Project\group-1.3


In [30]:
from scripts.utils import filter_comments_pandas
import pickle
import regex as re
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
processed_path = './processed'
with open(os.path.join(processed_path, 'processed_comments.pkl'), 'rb') as f:
    df_comments = pickle.load(f)
df_comments
# 135479 rows × 1 columns

,comment
0,That was another administration.
1,Why is the ex CEO of Monsanto appointed head o...
2,"because he greased the palms of our ""elected"" ..."
3,the photo shown above is from a feed lot in Ca...
4,THAT IS NOT GRASS FED MEET! NO GRASS THERE! YO...
...,...
135474,"Recognizing the power of Branding, the Amish s..."
135475,Amish tend to avoid 'branding' anything they m...
135476,"The Amish community around Lancaster, PA love ..."
135477,Look for Organic Valley dairy products. You ca...


In [26]:
next(df_comments.iterrows())[0]
next(df_comments.iterrows())[1]['comment']

0

how many comments include urls

In [32]:
url_idx_list = []
for x in df_comments.iterrows():
    idx = x[0]
    cont = x[1]['comment']
    if re.search(r'(http|https|www)\S+', cont):
        # print(cont)
        url_idx_list.append(idx)
len(url_idx_list)
# 5971

5971

clean comments, mainly remove urls

In [33]:
%%time
reprocessed_comment = filter_comments_pandas(df_comments)
# (135479, 1) -> (129307, 1)
# -> (130516, 1) after fixing issue of missing punctuation marks
# 17min

original length: (135479, 1) 	 length after filter out blank: (135479, 1)
removing Devanagari...
do some hard-coded replacement...
removing url...
calculating filtering criteria...
final dataframe length: (130516, 1)
Wall time: 16min 48s


In [36]:
reprocessed_comment.reset_index(drop=True, inplace=True)
# reprocessed_comment.rename(columns={'tidy_comment': 'comment'}, inplace=True)
# reprocessed_comment
reprocessed_comment['comment'].iloc[-21]

'Coincidentally, yesterday I just watched, "What the Dairy Industry does not Want You to Know" by Neal Barnard MD ( and that sure makes a strong case against dairy of any kind, organic or not. And I grew up on a farm too! I will stick to buying Amish furniture I guess. there is a place in my town that is called, "Amish Woodcrafts".'

In [37]:
# for comparison
df_comment['comment'].iloc[-21]

'Coincidentally, yesterday I just watched, "What the Dairy Industry does not Want You to Know" by Neal Barnard MD (https://www.youtube.com/watch?v=h3c_D0s391Q&amp;t=150s) and that sure makes a strong case against dairy of any kind, organic or not. And I grew up on a farm too! I will stick to buying Amish furniture I guess. there is a place in my town that is called, "Amish Woodcrafts".'

In [9]:
import os
# os.rename('./processed/processed_comments.pkl', './processed/processed_comments_orig.pkl')

In [38]:
with open(os.path.join(processed_path, 'processed_comments.pkl'), 'wb') as f:
    pickle.dump(reprocessed_comment, f)

In [46]:
reprocessed_comment.to_csv('../processed/processed_comments.csv', sep='\t')

In [39]:
with open(os.path.join(processed_path, 'processed_comments.pkl'), 'rb') as f:
    processed_comment = pickle.load(f)
processed_comment

,comment
0,That was another administration.
1,Why is the ex CEO of Monsanto appointed head o...
2,"because he greased the palms of our ""elected"" ..."
3,the photo shown above is from a feed lot in Ca...
4,THAT IS NOT GRASS FED MEET! NO GRASS THERE! YO...
...,...
130511,"Recognizing the power of Branding, the Amish s..."
130512,Amish tend to avoid 'branding' anything they m...
130513,"The Amish community around Lancaster, PA love ..."
130514,Look for Organic Valley dairy products. You ca...
